In [1]:
import sqlite3
import os
from csv import excel

from IPython.core.display_functions import clear_output
from exceptiongroup import catch
import mysql.connector
import re
from IPython.display import clear_output
import time


### current status: the extract function throws an error about cursors and hanlding them. probably an issue with transactions inside the sql dump, the python code has a bit of problem and if solved it could bump the accuracy up to around 50-60 percent.

p## MYSQL

### Functions and connections

In [2]:
mysql_config = {
    'host': '192.168.179.128',
    'user': 'a',
    'password': 'a',
    #DROP USW'database': 'mysql'  # Temporary database for processing
}
SPECIAL_CHARACTER='SSSSPECIAL_CHARRRRR'
EXTENTION_PGTOMYSQL='myfile'
class MySQLHandler(object):

    def __init__(self, config):
        self.config = config
        self.connection = mysql.connector.connect(**config)
        self.cursor = self.connection.cursor(buffered=True)

    def close(self):
        self.cursor.close()
        self.connection.close()
    def beat(self):
        self.close()
        self.connection = mysql.connector.connect(**mysql_config)
        self.cursor = self.connection.cursor(buffered=True)

    def remove_mysql_engine(self, the_stmt:str): # removes the mysql hardcode
        """
        Removes ENGINE=... (and optional AUTO_INCREMENT, CHARSET, COLLATE parts)
        from a MySQL CREATE TABLE statement string.
        """
        # Remove ENGINE=... plus optional AUTO_INCREMENT and table options
        # removing database charset comment on top
        stmt = re.sub(r'/\*![0-9]+\s.*?\*/', '', the_stmt,flags=re.IGNORECASE)
        # Matches ENGINE=InnoDB or ENGINE=MyISAM etc.
        stmt = re.sub(r'\sENGINE\s*=\s*\w+\b', '', stmt, flags=re.IGNORECASE)
        # Remove AUTO_INCREMENT=...
        stmt = re.sub(r'\sAUTO_INCREMENT\s*=\s*\d+\b', '', stmt, flags=re.IGNORECASE)
        # Remove DEFAULT CHARSET=... (if present)
        stmt = re.sub(r'\sDEFAULT\s+CHARSET\s*=\s*\w+\b', '', stmt, flags=re.IGNORECASE)
        # Remove COLLATE=...
        stmt = re.sub(r'\sCOLLATE\s*=\s*\w+\b', '', stmt, flags=re.IGNORECASE)
        # Clean up extra spaces before semicolon
        stmt = re.sub(r'\s*;\s*$', ';', stmt)
        return stmt.strip()

    def get_sql_content(self,address:str):
        _file= open(address,errors='ignore')
        _file_content = _file.read()
        _file.close()
        return _file_content

    def exec_all(self,script,special_divider_mode=False):
        _divider = SPECIAL_CHARACTER if special_divider_mode else ';'
        _response=[]
        for statement in script.split(_divider):
            if statement.strip():
                statement = statement+";"
                #print(statement)
                _result = self.cursor.execute(statement)
                _result = self.cursor.fetchall()
                _response.extend(_result)
        return _response

    def flush_sql(self,):
        _flush_queries = self.exec_all(self.get_sql_content(flusher_address))
        value = _flush_queries[0][0].decode("utf-8") if isinstance(_flush_queries[0][0],bytes) else _flush_queries[0][0]
        if value is not None:
            value = "SET FOREIGN_KEY_CHECKS = 0"+SPECIAL_CHARACTER+value+SPECIAL_CHARACTER+"SET FOREIGN_KEY_CHECKS = 1"
            self.exec_all(value,special_divider_mode=True)

    def extract_schema(self):
        response=""
        #The query for CREATING DBs
        _result = self.exec_all(self.get_sql_content(extractor_address))
        if len(_result) > 0: # rectify 0 database error
            _result = [_r[0] for _r in _result]
            _result = self.exec_all(SPECIAL_CHARACTER.join(_result),special_divider_mode=True)
            _result = [_r[1] for _r in _result]
            response =response+((';'.join(_result))+';\n')
        #The query for CREATING TABLEs
        _result = self.exec_all(self.get_sql_content(extractor_address2))
        if len(_result) > 0: # rectify 0 tables error
            _result = [_r[0] for _r in _result]
            _result=self.exec_all(SPECIAL_CHARACTER.join(_result),special_divider_mode=True)
            _result = [_r[1] for _r in _result]
            response = response+((';\n'.join(_result))+';\n')
        response=self.remove_mysql_engine(response)
        return response

    def create_temporary_database(self,):
        self.exec_all(self.get_sql_content(tempdb_address))


In [3]:
def pgtomysql(filename:str):
    os.system(f'./pg2mysql.pl <data/schemapile/sqlfiles_permissive/{filename}>data/schemapile/sqlfiles_permissive/{filename}{EXTENTION_PGTOMYSQL}')

In [4]:
import re

def getfieldname(line: str):
    """Extract the field name from a line of SQL."""
    line = line.strip()
    m = re.search(r'`([^`]*)`', line)
    if m:
        return m.group(1) or None
    m = re.match(r'([^\s]+)', line)
    if m:
        return m.group(1)
    return None


def pg2mysql_str(pg_sql: str) -> str:
    """
    Convert PostgreSQL dump string -> MySQL-compatible dump string.
    Removes Postgres-specific constructs like sequences, search_path, statement_timeout.
    """
    lines = pg_sql.splitlines(True)  # keep line endings
    output = 'SET SQL_MODE="NO_AUTO_VALUE_ON_ZERO";\nSET time_zone="+00:00";\n\n'

    in_create_table = False
    in_insert = False
    tbl_extra = ""
    values = []
    heads = None
    linenumber = 0

    while linenumber < len(lines):
        line = lines[linenumber].strip()

        # Skip Postgres-only commands
        if (line.startswith("SET statement_timeout") or
            line.startswith("SET search_path") or
            line.startswith("CREATE SEQUENCE")):
            linenumber += 1
            continue

        # CREATE TABLE
        if line.startswith("CREATE TABLE"):
            in_create_table = True
            line = line.replace('"', '`')
            output += line + "\n"
            linenumber += 1
            continue

        # End CREATE TABLE
        if line.startswith(");") and in_create_table:
            in_create_table = False
            output += tbl_extra + ");\n\n"
            tbl_extra = ""
            linenumber += 1
            continue

        # Inside CREATE TABLE
        if in_create_table:
            line = line.replace('"', '`')
            line = re.sub(r'\binteger\b', ' int(11)', line)
            line = line.replace(' serial ', ' int(11) auto_increment ')
            line = line.replace(' bigserial ', ' bigint auto_increment ')
            line = line.replace(' bytea', ' BLOB')
            line = line.replace(' boolean', ' bool')
            line = line.replace(' bool DEFAULT true', ' bool DEFAULT 1')
            line = line.replace(' bool DEFAULT false', ' bool DEFAULT 0')

            # character varying
            line = re.sub(r'character varying\((\d+)\)',
                          lambda m: f"varchar({m.group(1)})" if int(m.group(1)) <= 255 else "text", line)
            line = line.replace('character varying', 'varchar(255)')

            # character(n)
            line = re.sub(r'character\((\d+)\)',
                          lambda m: f"varchar({m.group(1)})" if int(m.group(1)) <= 255 else "text", line)

            # defaults
            line = re.sub(r"DEFAULT \('(\d*)'::(?:int|smallint|bigint)[^ ,]*", r" DEFAULT \1 ", line)
            line = re.sub(r"DEFAULT \(([-\d]*)\)", r" DEFAULT \1 ", line)
            line = re.sub(r" DEFAULT nextval\(.*\) ", " auto_increment ", line)
            line = re.sub(r"::[^,]*,", ",", line)
            line = re.sub(r"::.*$", "\n", line)

            # timestamps / time
            line = line.replace(" timestamp with time zone", " timestamp")
            line = line.replace(" timestamp without time zone", " timestamp")
            line = line.replace(" time with time zone", " time")
            line = line.replace(" time without time zone", " time")
            line = line.replace(" timestamp DEFAULT now()", " timestamp DEFAULT CURRENT_TIMESTAMP")

            # strip precision specifier (TIMESTAMP(3), TIME(6) → TIMESTAMP, TIME)
            line = re.sub(r'TIMESTAMP\(\d+\)', 'TIMESTAMP', line, flags=re.IGNORECASE)
            line = re.sub(r'TIME\(\d+\)', 'TIME', line, flags=re.IGNORECASE)

            # Add primary key if auto_increment
            if 'auto_increment' in line:
                field = getfieldname(line)
                if field:
                    tbl_extra += f", PRIMARY KEY(`{field}`)\n"

            # Reserved keywords in MySQL
            specialfields = {"repeat", "status", "type", "call"}
            field = getfieldname(line)
            if field and field in specialfields:
                line = line.replace(f"{field} ", f"`{field}` ", 1)

            # text fields with defaults not allowed
            if "text DEFAULT" in line:
                line = line.replace(" text DEFAULT ", " varchar(255) DEFAULT ")

            # drop constraints
            if " CONSTRAINT " in line:
                line = ""
                if output.endswith(",\n"):
                    output = output[:-2] + "\n"

            output += line + "\n"
            linenumber += 1
            continue

        # INSERT INTO (single or multi-line)
        if line.startswith("INSERT INTO"):
            before, sep, after = line.partition("VALUES")
            before = before.replace('"', '`')
            after = after.replace(" (E'", " ('").replace(", E'", ", '")
            output += before + sep + after
            linenumber += 1
            while not line.rstrip().endswith(");"):
                line = lines[linenumber]
                output += line.replace("', E'", "', '")
                linenumber += 1
            continue

        # ALTER TABLE ONLY → PRIMARY KEY
        if line.startswith("ALTER TABLE ONLY"):
            pkey = line.replace(" ONLY", "").replace('"', '`')
            linenumber += 1
            if linenumber < len(lines) and " PRIMARY KEY " in lines[linenumber]:
                output += pkey + lines[linenumber]
            linenumber += 1
            continue

        # CREATE INDEX
        if line.startswith("CREATE INDEX"):
            m = re.match(r'CREATE INDEX "?([a-zA-Z0-9_]*)"? ON "?([a-zA-Z0-9_]*)"? USING btree \((.*)\);', line)
            if m:
                output += f"ALTER TABLE `{m.group(2)}` ADD INDEX ( {m.group(3)} ) ;\n"
            linenumber += 1
            continue

        # DATABASE ops
        if line.startswith("DROP DATABASE"):
            output += line + "\n"
            linenumber += 1
            continue

        if line.startswith("CREATE DATABASE"):
            m = re.match(r"CREATE DATABASE ([a-zA-Z0-9_]*) .* ENCODING = '(.*)'", line)
            if m:
                output += f"CREATE DATABASE `{m.group(1)}` DEFAULT CHARACTER SET {m.group(2)};\n\n"
            linenumber += 1
            continue

        if line.startswith("\\connect"):
            m = re.search(r'connect ([a-zA-Z0-9_]*)', line)
            if m:
                output += f"USE `{m.group(1)}`;\n\n"
            linenumber += 1
            continue

        # COPY → INSERT
        if line.startswith("COPY "):
            m = re.match(r'COPY (.*) FROM stdin', line)
            if m:
                heads = m.group(1).replace('"', '`')
                values = []
                in_insert = True
            linenumber += 1
            continue

        if in_insert:
            if line.strip() == "\\.":
                in_insert = False
                if values:
                    output += f"INSERT INTO {heads} VALUES\n" + ",\n".join(values) + ";\n\n"
                values = []
            else:
                vals = line.rstrip("\n").split("\t")
                vals = [("NULL" if v == "\\N" else "'" + v.replace("'", "\\'") + "'") for v in vals]
                values.append("(" + ",".join(vals) + ")")
                if len(values) >= 1000:
                    output += f"INSERT INTO {heads} VALUES\n" + ",\n".join(values) + ";\n"
                    values = []
            linenumber += 1
            continue

        # fallback
        output += line + "\n"
        linenumber += 1

    return output


In [5]:
flusher_address = "data/flush_mysql.sql"
extractor_address = "data/extract_schema.sql"
extractor_address2 = "data/extract_tables.sql"
tempdb_address = "data/temporary_database.sql"

dataset_directory_address="data/schemapile/sqlfiles_permissive/"
purified_dataset_directory_address="data/pure_schemapile/sqlfiles_permissive/"
erroneous=0
correct=0

dataset_directory = os.listdir(dataset_directory_address)

for _file_name in dataset_directory:
    _pure_content=None
    sqlhandler=MySQLHandler(mysql_config)
    print(_file_name)
    sqlhandler.flush_sql()
    sqlhandler.create_temporary_database()
    content = sqlhandler.get_sql_content(dataset_directory_address+_file_name)
    try: # try running on mysql first
        sqlhandler.exec_all(content) # Changing mysql databases based on the sql file
    except:
        sqlhandler.beat()
        sqlhandler.flush_sql()
        sqlhandler.create_temporary_database()
        try:
            #content = pg2mysql_str(content)
            pgtomysql(_file_name) # mysql didn't work, so convert to postgres
            content = sqlhandler.get_sql_content(dataset_directory_address+_file_name+EXTENTION_PGTOMYSQL) # read the converted content
            sqlhandler.exec_all(content)
        except Exception as e:
            print(e)
    finally:
        sqlhandler.beat()
        _pure_content= sqlhandler.extract_schema()
        if _pure_content is not None and _pure_content!="":
            _pure_file = open(purified_dataset_directory_address+_file_name,mode="w+",errors="ignore") # Writing standardized mysql databases inside a sql file
            _pure_file.writelines(_pure_content)
            _pure_file.flush()
            _pure_file.close()
            correct+=1
        else:
            erroneous+=1
        sqlhandler.close()


    clear_output(wait=True)
    display(None)
    print(f"Current/Total: {correct+erroneous}/{len(dataset_directory)}")
    print(f"Correct: {correct}")
    print(f"Errors: {erroneous}")
    print(f"Proportion: {round(correct/(erroneous+correct),3)}")


print('REPORT:\n')
print(f'erroneous:{erroneous}, {erroneous/(correct+erroneous)}')
print('correct:',correct)
print('total:',correct+erroneous)

Current/Total: 40805/40805
Correct: 26312
Errors: 14493
Proportion: 0.645
REPORT:

erroneous:14493, 0.3551770616346036
correct: 26312
total: 40805


In [ ]:
from IPython.display import clear_output
import time

for i in range(5):
    clear_output(wait=True)
    display(None)
    # clear previous output
    print(f"Line 1: {i}")
    print(f"Line 2: {i*2}")
    print(f"Line 3: {i*3}")


d## SQLITE

In [ ]:
import os
dataset_directory_address="data/schemapile/sqlfiles_permissive/"
purified_dataset_directory_address="data/pure_schemapile/sqlfiles_permissive/"
erroneous=0
correct=0

dataset_directory = os.listdir(dataset_directory_address)
for _file_name in dataset_directory:
    try:
        _file= open(dataset_directory_address + _file_name)
        _file_content = _file.read()
        connection = sqlite3.connect(":memory:")
        cur = connection.cursor()
        #cur.execute("PRAGMA foreign_keys = ON;")
        cur.executescript(_file_content)
        response = cur.execute("SELECT sql FROM sqlite_master WHERE type='table';")
        #print(response.fetchall())
        _pure_content = [p[0] for p in response.fetchall()] # returns a tuple, first is the content, second is the result that is not needed
        _pure_file = open(purified_dataset_directory_address+_file_name,mode="w+")
        _pure_file.writelines(_pure_content)
        _pure_file.flush()
        _pure_file.close()
        cur.close()
        _file.close()
        correct+=1
    except:
        erroneous+=1
print('REPORT:\n')
print(f'erroneous:{erroneous}, {erroneous/(correct+erroneous)}')
print('correct:',correct)
print('total:',correct+erroneous)

In [ ]:
def remove_mysql_engine(the_stmt:str): # removes the mysql hardcode
    """
    Removes ENGINE=... (and optional AUTO_INCREMENT, CHARSET, COLLATE parts)
    from a MySQL CREATE TABLE statement string.
    """
    # Remove ENGINE=... plus optional AUTO_INCREMENT and table options
    # Matches ENGINE=InnoDB or ENGINE=MyISAM etc.
    stmt = re.sub(r'\sENGINE\s*=\s*\w+\b', '', the_stmt, flags=re.IGNORECASE)
    # Remove AUTO_INCREMENT=...
    stmt = re.sub(r'\sAUTO_INCREMENT\s*=\s*\d+\b', '', stmt, flags=re.IGNORECASE)
    # Remove DEFAULT CHARSET=... (if present)
    stmt = re.sub(r'\sDEFAULT\s+CHARSET\s*=\s*\w+\b', '', stmt, flags=re.IGNORECASE)
    # Remove COLLATE=...
    stmt = re.sub(r'\sCOLLATE\s*=\s*\w+\b', '', stmt, flags=re.IGNORECASE)
    # Clean up extra spaces before semicolon
    stmt = re.sub(r'\s*;\s*$', ';', stmt)
    return stmt.strip()

In [ ]:
st="""CREATE TABLE `CMIResults` (
  `userID` int NOT NULL,
  `courseID` int DEFAULT NULL,
  `moduleID` int DEFAULT NULL,
  `SCORMCMIData` text,
  `startedDate` varchar(32) DEFAULT NULL,
  `status` varchar(32) DEFAULT NULL,
  `completedDate` varchar(32) DEFAULT NULL,
  `score` varchar(32) DEFAULT NULL,
  `scoreRaw` decimal(10,2) DEFAULT '0.00',
  PRIMARY KEY (`userID`)
) ENGINE=InnoDB DEFAULT CHARSET=latin1;"""

print(remove_mysql_engine(st))

In [ ]:
import re
import time




In [ ]:
pgsql="""-- CreateTable
CREATE TABLE "conclusions" (
    "id" VARCHAR(50) NOT NULL,
    "vaccinated_at" TIMESTAMP(3) NOT NULL DEFAULT CURRENT_TIMESTAMP,
    "appointment_id" VARCHAR(50) NOT NULL,
    "vaccinated_by_id" VARCHAR(50) NOT NULL,
    "vaccine_batch_id" VARCHAR(50) NOT NULL,

    PRIMARY KEY ("id")
);

-- AddForeignKey
ALTER TABLE "conclusions" ADD FOREIGN KEY ("appointment_id") REFERENCES "appointments"("id") ON DELETE CASCADE ON UPDATE CASCADE;

-- AddForeignKey
ALTER TABLE "conclusions" ADD FOREIGN KEY ("vaccinated_by_id") REFERENCES "health_professionals"("id") ON DELETE CASCADE ON UPDATE CASCADE;

-- AddForeignKey
ALTER TABLE "conclusions" ADD FOREIGN KEY ("vaccine_batch_id") REFERENCES "vaccine_batches"("id") ON DELETE CASCADE ON UPDATE CASCADE;
"""

converted = pg2mysql_str(pgsql)


In [ ]:
print(converted)

In [ ]:
import os

In [ ]:
os.system('./pg2mysql.pl <data/schemapile/sqlfiles_permissive/001590_098%20Room%20Feature%20Type.sql>halo.sql')